In [ ]:
# !pip install tensorflow tensorflow-hub
# !pip install librosa soundfile
# !pip install scikit-learn pandas numpy matplotlib seaborn

Mounted at /content/drive


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import glob
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

2025-07-24 13:03:44.521262: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-24 13:03:44.543447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753355024.560994  272417 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753355024.567335  272417 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753355024.581277  272417 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
PERCH_MODEL_URL = "https://www.kaggle.com/models/google/bird-vocalization-classifier/TensorFlow2/bird-vocalization-classifier/8"

try:
    perch_model = hub.load(PERCH_MODEL_URL)
    print(" PERCH loaded successfully!")
except Exception as e:
    print(f"Error loading PERCH: {e}")

2025-07-24 13:04:02.648510: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


 PERCH loaded successfully!


In [3]:
import tarfile
import pickle

In [4]:
def extract_tar_files(tar_path, extract_to):
    print(f"Extracting {tar_path}...")
    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(extract_to)
    print(f"Extracted to {extract_to}")


In [ ]:
def load_spectrogram_dataset(data_directory):

    extract_dir = os.path.join(data_directory, "extracted")

    # Check if already extracted
    expected_subdir = os.path.join(extract_dir, "npy_spectrograms")

    if os.path.exists(expected_subdir) and os.listdir(expected_subdir):
        print(f"Dataset already extracted at {extract_dir}")
        print("Skipping extraction...")

        print("Extracted contents:")
        for root, dirs, files in os.walk(extract_dir):
            for file in files[:10]:
                print(f"  {os.path.join(root, file)}")
            if len(files) > 10:
                print(f"  ... and {len(files) - 10} more files")

        return extract_dir

    print("Dataset not found or empty. Proceeding with extraction...")
    os.makedirs(extract_dir, exist_ok=True)

    tar_files = [
        "npy_spectrograms.tar.gz"
    ]

    for tar_file in tar_files:
        tar_path = os.path.join(data_directory, tar_file)
        if os.path.exists(tar_path):
            extract_tar_files(tar_path, extract_dir)
        else:
            print(f"Warning: {tar_file} not found at {tar_path}")

    print("Extracted contents:")
    for root, dirs, files in os.walk(extract_dir):
        for file in files[:10]:
            print(f"  {os.path.join(root, file)}")
        if len(files) > 10:
            print(f"  ... and {len(files) - 10} more files")

    return extract_dir

In [6]:
def load_npy_spectrograms(extracted_dir):
    spectrograms = []
    labels = []
    filepaths = []

    npy_files = []
    for root, dirs, files in os.walk(extracted_dir):
        for file in files:
            if file.endswith('.npy'):
                npy_files.append(os.path.join(root, file))

    print(f"Found {len(npy_files)} .npy files")

    for npy_file in tqdm(npy_files, desc="Loading spectrograms"):
        try:
            spectrogram = np.load(npy_file)
            spectrograms.append(spectrogram)

            filename = os.path.basename(npy_file)
            if '_' in filename:
                label = filename.split('_')[0]
            else:
                label = filename.replace('.npy', '')

            labels.append(label)
            filepaths.append(npy_file)

        except Exception as e:
            print(f"Error loading {npy_file}: {e}")
            continue

    print(f"Loaded {len(spectrograms)} spectrograms")
    print(f"Species distribution:")
    label_counts = pd.Series(labels).value_counts()
    print(label_counts.head(10))

    return spectrograms, labels, filepaths

In [7]:
# def spectrogram_to_audio_representation(spectrogram, sample_rate = 32000):
#     try:
#         target_length = sample_rate * 5

#         if len(audio_representation) < target_length:
#             pad_amount = target_length - len(audio_representation)
#             pad_left = pad_amount // 2
#             pad_right = pad_amount - pad_left
#             audio_representation = np.pad(audio_representation,
#                                         (pad_left, pad_right),
#                                         mode='constant')
#         else:
#             audio_representation = audio_representation[:target_length]

#         # Normalize
#         if np.max(np.abs(audio_representation)) > 0:
#             audio_representation = audio_representation / np.max(np.abs(audio_representation)) * 0.25

#         return audio_representation.astype(np.float32)

#     except Exception as e:
#         print(f"Error converting spectrogram: {e}")
#         return None

In [8]:
def spectrogram_to_audio_representation(spectrogram, sample_rate=44100, hop_size=512):
    try:
        if len(spectrogram.shape) == 2:
            audio_representation = librosa.istft(spectrogram)
        else:
            audio_representation = spectrogram

        target_length = 5 * sample_rate

        if len(audio_representation) < target_length:
            audio_representation = np.pad(audio_representation,
                                        (0, target_length - len(audio_representation)),
                                        mode='constant')
        else:
            audio_representation = audio_representation[:target_length]

        if np.max(np.abs(audio_representation)) > 0:  # normalize
            audio_representation = audio_representation / np.max(np.abs(audio_representation)) * 0.25

        return audio_representation.astype(np.float32)
    except Exception as e:
        print(f"Error converting spectrogram: {e}")
        return None

In [9]:
# PERCH Feature Extraction
def extract_perch_embeddings_from_spectrograms(spectrograms, labels, batch_size=32):
    """
    Extract embeddings using PERCH model from spectrograms
    """

    embeddings = []
    valid_labels = []

    print("Converting spectrograms to audio representations and extracting PERCH embeddings")

    for i in tqdm(range(0, len(spectrograms), batch_size)):
        batch_spectrograms = spectrograms[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]

        batch_audio = []
        batch_valid_labels = []

        # Convert spectrograms to audio representations
        for spectrogram, label in zip(batch_spectrograms, batch_labels):
            audio_repr = spectrogram_to_audio_representation(spectrogram)
            if audio_repr is not None:
                batch_audio.append(audio_repr)
                batch_valid_labels.append(label)

        if batch_audio:
            # Converting to tensor
            batch_tensor = tf.stack(batch_audio)

            try:
                batch_embeddings = perch_model.infer_tf(batch_tensor)

                embeddings.extend(batch_embeddings['embedding'].numpy())

                valid_labels.extend(batch_valid_labels)

            except Exception as e:
                print(f"Error processing batch: {e}")
                continue

    print(f"Successfully extracted {len(embeddings)} embeddings")
    return np.array(embeddings), valid_labels

In [10]:
# Alt approach: Using spectrograms directly with a different model  # will not use
def extract_features_from_spectrograms_directly(spectrograms, labels):
    """
    Extract features directly from spectrograms
    """

    features = []
    valid_labels = []

    print("Extracting features directly from spectrograms.")

    for spectrogram, label in tqdm(zip(spectrograms, labels), desc="Processing spectrograms"):
        try:
            spec_features = []

            # Ensure 2D spectrogram
            if len(spectrogram.shape) == 1:
                # If 1D, reshape to 2D
                spectrogram = spectrogram.reshape(-1, 1)

            # Feature 1: Spectral centroid (mean frequency)
            spectral_centroid = np.mean(spectrogram, axis=1)
            spec_features.extend([np.mean(spectral_centroid), np.std(spectral_centroid)])

            # Feature 2: Spectral rolloff
            spectral_rolloff = np.percentile(spectrogram, 85, axis=1)
            spec_features.extend([np.mean(spectral_rolloff), np.std(spectral_rolloff)])

            # Feature 3: Zero crossing rate equivalent (variation across time)
            if spectrogram.shape[1] > 1:
                zcr_equiv = np.mean(np.abs(np.diff(spectrogram, axis=1)), axis=1)
                spec_features.extend([np.mean(zcr_equiv), np.std(zcr_equiv)])

            # Feature 4: Spectral bandwidth
            spec_bandwidth = np.std(spectrogram, axis=1)
            spec_features.extend([np.mean(spec_bandwidth), np.std(spec_bandwidth)])

            # Feature 5: Statistical moments
            spec_features.extend([
                np.mean(spectrogram),
                np.std(spectrogram),
                np.max(spectrogram),
                np.min(spectrogram)
            ])

            # Feature 6: Frequency bin energies (first 20 bins)
            freq_energies = np.mean(spectrogram[:min(20, spectrogram.shape[0]), :], axis=1)
            spec_features.extend(freq_energies.tolist())

            features.append(spec_features)
            valid_labels.append(label)

        except Exception as e:
            print(f"Error processing spectrogram: {e}")
            continue

    print(f"Extracted features from {len(features)} spectrograms")
    return np.array(features), valid_labels

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

Pipeline

In [ ]:
metadata_csv_path = '/home/ldrich/Summer2025BHT/deep_learning/project_bird/data/metadata/metadata.csv'
spectrogram_base_dir = '/home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio'

In [17]:
extracted_dir = load_spectrogram_dataset(spectrogram_base_dir)

if not os.path.isdir(extracted_dir):
    print(f"Error: Extraction directory '{extracted_dir}' does not exist. Aborting.")

if not os.path.exists(metadata_csv_path):
    print(f"Error: Metadata CSV not found at '{metadata_csv_path}'. Aborting.")

metadata_df = pd.read_csv(metadata_csv_path)
print(f"Loaded metadata from {metadata_csv_path}. Total entries: {len(metadata_df)}")
print("Metadata columns found:", metadata_df.columns.tolist())

Dataset not found or empty. Proceeding with extraction...
Extracting /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/npy_spectrograms.tar.gz...
Extracted to /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/extracted
Extracted contents:
  /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/extracted/npy_spectrograms/Chestnut-vented Warbler/Chestnut-vented_Warbler_34.npy
  /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/extracted/npy_spectrograms/Chestnut-vented Warbler/Chestnut-vented_Warbler_53.npy
  /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/extracted/npy_spectrograms/Chestnut-vented Warbler/Chestnut-vented_Warbler_46.npy
  /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audio/extracted/npy_spectrograms/Chestnut-vented Warbler/Chestnut-vented_Warbler_36.npy
  /home/ldrich/Summer2025BHT/deep_learning/project_bird/data/processed_audi

In [ ]:
metadata_df.head(5)

,species,npy_filename,original_mp3,duration,split
0,African Paradise Flycatcher,African_Paradise_Flycatcher_01.npy,../data/raw_audio/sa_bird_audio/African Paradi...,17.868390,train
1,African Paradise Flycatcher,African_Paradise_Flycatcher_02.npy,../data/raw_audio/sa_bird_audio/African Paradi...,28.732971,train
2,African Paradise Flycatcher,African_Paradise_Flycatcher_03.npy,../data/raw_audio/sa_bird_audio/African Paradi...,19.489002,test
3,African Paradise Flycatcher,African_Paradise_Flycatcher_04.npy,../data/raw_audio/sa_bird_audio/African Paradi...,28.953560,train
4,African Paradise Flycatcher,African_Paradise_Flycatcher_05.npy,../data/raw_audio/sa_bird_audio/African Paradi...,29.000000,test


In [ ]:
# Constructing full paths to the .npy files
metadata_df['full_npy_path'] = metadata_df.apply(
        lambda row: os.path.join(extracted_dir, 'npy_spectrograms', row['species'], row['npy_filename']),
        axis=1
    ) # because of the folder structure

In [ ]:
metadata_df.head(5)

,species,npy_filename,original_mp3,duration,split,full_npy_path
0,African Paradise Flycatcher,African_Paradise_Flycatcher_01.npy,../data/raw_audio/sa_bird_audio/African Paradi...,17.868390,train,/content/drive/MyDrive/BHT/Projects/Deep Learn...
1,African Paradise Flycatcher,African_Paradise_Flycatcher_02.npy,../data/raw_audio/sa_bird_audio/African Paradi...,28.732971,train,/content/drive/MyDrive/BHT/Projects/Deep Learn...
2,African Paradise Flycatcher,African_Paradise_Flycatcher_03.npy,../data/raw_audio/sa_bird_audio/African Paradi...,19.489002,test,/content/drive/MyDrive/BHT/Projects/Deep Learn...
3,African Paradise Flycatcher,African_Paradise_Flycatcher_04.npy,../data/raw_audio/sa_bird_audio/African Paradi...,28.953560,train,/content/drive/MyDrive/BHT/Projects/Deep Learn...
4,African Paradise Flycatcher,African_Paradise_Flycatcher_05.npy,../data/raw_audio/sa_bird_audio/African Paradi...,29.000000,test,/content/drive/MyDrive/BHT/Projects/Deep Learn...


In [ ]:
# Filter metadata for files that exist on drive
initial_count = len(metadata_df)
metadata_df = metadata_df[metadata_df['full_npy_path'].apply(os.path.exists)]
if len(metadata_df) < initial_count:
    print(f"Warning: {initial_count - len(metadata_df)} .npy files referenced in metadata were not found on disk. These will be skipped.")

if metadata_df.empty:
    print("No valid .npy files found after checking existence. Aborting.")

In [ ]:
# Unique class names from the metadata
all_class_names = sorted(metadata_df['species'].unique().tolist())
print(f"Detected {len(all_class_names)} unique species from metadata.")

Detected 40 unique species from metadata.


In [ ]:
# Load all spectrograms, their labels, and their original file paths
all_spectrograms, all_labels_from_npy, all_npy_paths_loaded = load_npy_spectrograms(extracted_dir)

Found 1768 .npy files


Loading spectrograms:  55%|█████▍    | 970/1768 [14:00<10:28,  1.27it/s]

In [ ]:
# For loaded NPYs to easily link with metadata
loaded_data_df = pd.DataFrame({
    'spectrogram': all_spectrograms,
    'label_from_npy': all_labels_from_npy,
    'full_npy_path': all_npy_paths_loaded
})

# Merge loaded NPY data with metadata to get 'split' column
# This aligns the loaded spectrograms/labels with their split information
processed_data_df = pd.merge(
    loaded_data_df,
    metadata_df[['full_npy_path', 'species', 'split']], # Keep 'species' from metadata for accuracy
    on='full_npy_path',
    how='inner',
    suffixes=('_npy', '_metadata') # Distinguish if needed, but 'species' should be consistent
)

processed_data_df['final_label'] = processed_data_df['species']

if processed_data_df.empty:
    print("No data matched between loaded NPYs and metadata. Aborting.")
print(f"Matched {len(processed_data_df)} samples with metadata for processing.")

In [ ]:
processed_data_df.head(5)

In [ ]:
# Feature extraction
try:
    print("Attempting PERCH feature extraction.")
    embeddings, valid_labels_for_embeddings = extract_perch_embeddings_from_spectrograms(
        processed_data_df['spectrogram'].tolist(),
        processed_data_df['final_label'].tolist(),
        batch_size=16
    )
    feature_method = "PERCH"
except Exception as e:
    print(f"PERCH extraction failed: {e}")
    print("Falling back to direct spectrogram feature extraction.")
    embeddings, valid_labels_for_embeddings = extract_features_from_spectrograms_directly(
        processed_data_df['spectrogram'].tolist(),
        processed_data_df['final_label'].tolist()
    )
    feature_method = "Direct Spectrogram Features"

if len(embeddings) == 0:
    print("No features extracted. Aborting.")
else:
    print(f"Feature extraction complete using {feature_method}. Feature shape: {embeddings.shape}")

In [ ]:
embeddings_data = pd.DataFrame(embeddings)
embeddings_data['original_label'] = valid_labels_for_embeddings

# It's important that the order of `embeddings` corresponds to `processed_data_df`
# if `extract_perch_embeddings_from_spectrograms` handles skipping.
# Assuming `extract_perch_embeddings_from_spectrograms` maintains order of successful extractions.
# We need to re-align the embeddings with the split column.
embeddings_data['full_npy_path'] = processed_data_df['full_npy_path'].iloc[:len(embeddings_data)].values # Re-align paths

In [ ]:
final_processed_df = pd.merge(
    embeddings_data,
    metadata_df[['full_npy_path', 'split']], # Merge with original metadata for split info
    on='full_npy_path',
    how='inner'
)
if final_processed_df.empty:
    print("Error: Final data for splitting is empty after merging embeddings with splits.")

In [ ]:
final_processed_df.head(5)

In [ ]:
# Split data based on 'split' column in the metadata
X_train_df = final_processed_df[final_processed_df['split'] == 'train']
X_val_df = final_processed_df[final_processed_df['split'] == 'val']
X_test_df = final_processed_df[final_processed_df['split'] == 'test']

# Extract features (X) and labels (y)
feature_cols = [col for col in final_processed_df.columns if isinstance(col, (int, float))]  # selecting columns by type
X_train = X_train_df[feature_cols].values
y_train_labels = X_train_df['original_label'].tolist()

X_val = X_val_df[feature_cols].values
y_val_labels = X_val_df['original_label'].tolist()

X_test = X_test_df[feature_cols].values
y_test_labels = X_test_df['original_label'].tolist()

print(f"Train samples after split: {len(X_train)}")
print(f"Validation samples after split: {len(X_val)}")
print(f"Test samples after split: {len(X_test)}")

# Check for empty splits
if len(X_train) == 0 or len(X_val) == 0:
    print("Error: One or more data splits are empty. Please check your metadata CSV and NPY files.")